In [ ]:
# Importing Libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

%matplotlib inline

# Creating Prediction Model based on existing Clients

In [ ]:
# Reading the training data

df = pd.read_csv('Credit_data.csv')
df.T.head(50)

In [ ]:
# Inspecting the data

print(df.shape)
print('----------------')
print(df.describe())
print('----------------')
print(df.info())

## Pre-prossesing the data

- After inspection in the data it can be noticed that in the column EDUCATION the values for Unknown info are represented with 3 different numbers: 0,5,6. In order to reduce the number of categories the values of 5 and 6 were replaced by 0.
- Also the column SEX provides a binary information about the Gender of the client, taking Values 1, 2. In order for the values to be in range 0 to 1 without losing any information, the value 1 was subtracted. Now, Males are represented with 0 and Females with 1.

In [ ]:
# Replacing the values of 5,6 in Education column with 0 because they denote the same Unknown info 

print('Before replace:' , df['EDUCATION'].unique())
df['EDUCATION'].replace({5 : 0, 6 : 0}, inplace=True)
print('After replace:', df['EDUCATION'].unique())

# Making the values of Sex 0 and 1
print('Before reduction:' , df['SEX'].unique())
df['SEX'] = df['SEX']- 1
print('After reduction:', df['SEX'].unique())

In columns PAY_1 to PAY_6 represented the repayment status of each client from 1 to 6 months prior to October respectively. It can be seen that the values from -2 to 0 represent different categories of clients and form 1 to 9 represent the months that a client delay hos payments. To separate then the categorical values from the numerical, two new columns were created for each column PAY.
- PAY_CATEGORY represents the same categorical values from -2 to 0. In addition, a new category was add from the clients that were delayed. These clients are represented with the value of 1.
- DELAY represents the months of delay for each client, captured by the numerical values from 1 to 9. Moreover, the value 0 was added for the non - delayed clients (with values -2, -1, 0).  

In [ ]:
# Separating the categorical, and numirical values in columns PAY

for i in range(1,7):
    column = 'PAY_'+str(i)
    ncol = 'DELAY_'+str(i)

    col_num = df[column]
    col_num = col_num.replace({-2: 0, -1: 0})
    NUM = pd.DataFrame(list(col_num), columns=[ncol])
    col_cat = df[column]
    col_cat = col_cat.replace({2: 1, 3: 1, 4: 1, 5: 1, 6: 1, 7: 1, 8: 1, 9: 1})
    CAT = pd.DataFrame(list(col_cat), columns=[column+'_CATEGORY'])
    if i==1:
        dfNew = pd.concat([NUM,CAT], axis=1)
    else:
        temp = pd.concat([NUM,CAT], axis=1)
        dfNew = pd.concat([dfNew, temp], axis=1)

df = pd.concat([df, dfNew], axis=1)
df = df.drop(['PAY_1','PAY_2','PAY_3','PAY_4','PAY_5','PAY_6'], axis=1)

- For the columns having more than two or more categorical values, dummy variables were created.

In [ ]:
# Creating Dummies for categorical variables

df = pd.get_dummies(df,drop_first=True,columns=['EDUCATION','MARRIAGE'])
df = pd.get_dummies(df,drop_first=True,columns=['PAY_1_CATEGORY','PAY_2_CATEGORY','PAY_3_CATEGORY',
                                                'PAY_4_CATEGORY','PAY_5_CATEGORY','PAY_6_CATEGORY'])

In [ ]:
df.shape

In [ ]:
# Searching for missing values

df.isnull().sum()

- One of the basic assumptions that must be met for logistic regression is the absence of multicollinearity. For these reason highly correlated features need to be excluded from the data.

In [ ]:
# Checking for high correlation between the variables

plt.figure(figsize=(40,20))

correlation_matrix = df.corr()

mask = np.zeros_like(correlation_matrix, dtype=bool)
mask[np.triu_indices_from(mask)] = True

sns.heatmap(correlation_matrix, annot=True, mask=mask)
plt.show()

In [ ]:
# Excluding highly correlated Variables

ex_var = ['BILL_AMT2','BILL_AMT4','BILL_AMT6','BILL_AMT3',
          'MARRIAGE_2','PAY_2_CATEGORY_1','PAY_3_CATEGORY_1',
          'PAY_6_CATEGORY_1','PAY_4_CATEGORY_1','PAY_5_CATEGORY_1',
          'BILL_AMT6','BILL_AMT5','PAY_1_CATEGORY_1','PAY_2_CATEGORY_0']

In [ ]:
temp = df.drop(ex_var, axis=1)

In [ ]:
# Checking if there is any high correlation between the variables after the exclution.

plt.figure(figsize=(30,15))

correlation_matrix = temp.corr()

mask = np.zeros_like(correlation_matrix, dtype=bool)
mask[np.triu_indices_from(mask)] = True

sns.heatmap(correlation_matrix, annot=True, mask=mask)
plt.show()

In [ ]:
# Updating the dataset 
df = temp

In [ ]:
df.T.head(50)

## Building the model

In [ ]:
# Taking as indepentent variables all the Variables exept from ID and defualt_0

Y = df[['default_0']]
X = df.drop(['ID','default_0'], axis=1)

In [ ]:
# Spliting the data to test set and other (test 20%)
TrainX, OtherX, TrainY, OtherY = train_test_split(X,Y, test_size=0.40, random_state=567)

# Spliting the data to training and validation set 
ValidationX, TestX, ValidationY, TestY = train_test_split(OtherX, OtherY, test_size=0.50, random_state=567)

In [ ]:
print('Percentage of Default in the Train set:', np.count_nonzero(TrainY)/len(TestY)) 
print('Percentage of Default in the Validation set:', np.count_nonzero(ValidationY)/len(TestY)) 
print('Percentage of Default in the Test set:', np.count_nonzero(TestY)/len(TestY)) 

In [ ]:
# Normalizing the columns for the continuous Variables

col_norm = ['LIMIT_BAL', 'AGE', 'PAY_AMT1', 'PAY_AMT2','PAY_AMT3','PAY_AMT4','PAY_AMT5','PAY_AMT6',
             'BILL_AMT1','DELAY_1','DELAY_2','DELAY_3','DELAY_4','DELAY_5','DELAY_6']

scaler = preprocessing.MinMaxScaler()
TrainX[col_norm] = scaler.fit_transform(TrainX[col_norm])

In [ ]:
TrainX.T.head(50)

In [ ]:
# Fitting the training set in the Logistic Regression 

logr = LogisticRegression(max_iter=1000).fit(TrainX, TrainY.values.ravel())
print("The model's Coefficients are:", logr.coef_)
print("The model's R^2 is:", logr.score(TrainX, TrainY))

In [ ]:
# Normalizing the columns with different orders of magnitude in the validation set 

scaler = preprocessing.MinMaxScaler()
ValidationX[col_norm] = scaler.fit_transform(ValidationX[col_norm])

In [ ]:
# Calculating the probabilities based on the validation set

Y_probs = logr.predict_proba(ValidationX)[:,1]
Y_probs

In [ ]:
# Creating the ROC curve

fpr, tpr, thresholds = roc_curve(ValidationY, Y_probs)

# Dispaying the ROC curve

fig = plt.figure()
ax = fig.add_subplot(1,1,1)
ax.plot(fpr, tpr, linewidth=3)
plt.show()

# Finding the AUC score of the curve

AUC = roc_auc_score(ValidationY, Y_probs)
print('The AUC score of the ROC curve is:', AUC) 

## Selecting a Threshold 

Due to the description of the assignment it can be seen that the predictions of the model had an economic impact on the bank. More specifically:
- For each new applicant that receive the credit and repaid in full the bank gains £1.500.  
- For each new applicant that receive the credit and failed to repaid default the bank losses £5000. 

Based on these info, the profit for each threshold can be calculated and the threshold that maximize that profits need to be selected.

In [ ]:
# Finding the thresholds that maximize the profits

prof=[]
for thres in thresholds:
    
    # Find the predictions upper of these theshold
    Y_pred = np.where(Y_probs > thres, 1, 0)
    cm = confusion_matrix(ValidationY,Y_pred)
    TN = cm[0][0]
    FP = cm[0][1]
    FN = cm[1][0]
    TP = cm[1][1]
    accur = (TN+TP)/(TN+TP+FN+TP)
    sens = (TP)/(TP+FN)
    spec = (TN)/(TN+FP)
    prec = (TP)/ (TP+FP)
    # Calculate the profit for these thresholds 
    profit= TN*1500 + FP*0 - FN*5000 + TP*0
    prof.append([thres, profit, accur, sens, spec, prec])

# Sorting the data based on thershold in order to visualisise
prof.sort(key = lambda x: x[0])
thres_df = pd.DataFrame(prof, columns=['Threshold','Profit','Accuracy','Sensitivity','Specificity','Precision'])
thres_df.head()

In [ ]:
# Visualizing the profits along with Accuracy, Sensitivity and Specificity for each threshold

fig1 = plt.figure(figsize=(12,5))
ax1 = fig1.add_subplot(1,2,1)
ax2 = fig1.add_subplot(1,2,2)
thres_df.plot.line(x='Threshold', y=['Profit'], ax=ax1)
thres_df.plot.line(x='Threshold' ,y=['Accuracy','Sensitivity','Specificity'], ax=ax2)
plt.show()

In [ ]:
# Choosing the threshold that generate the most profit

prof.sort(key = lambda x: x[1], reverse=True)
chosen_threshold = prof[0][0]
print(chosen_threshold)
threshold_idx = np.where(thresholds == chosen_threshold)[0][0]
print(threshold_idx)

In [ ]:
# Finding the FPR and TPR for this Threshold:
print("At threshold  " + str(thresholds[threshold_idx]))
print("the False Positive rate is " + str(fpr[threshold_idx]))
print("the True Positive rate is " + str(tpr[threshold_idx]))

In [ ]:
# Finding the predictions based on chosen threshold
Y_pred = np.where(Y_probs > chosen_threshold, 1, 0)
# Creating the confusion matrix
cm = confusion_matrix(ValidationY,Y_pred)
TN = cm[0][0]
FP = cm[0][1]
FN = cm[1][0]
TP = cm[1][1]

# Visualizing the Confusion Matrix
fig1 = plt.figure()
ax1 = fig.add_subplot(1,1,1)
sns.heatmap(pd.DataFrame(cm), annot=True, cmap="YlGnBu" ,fmt='g')
plt.ylabel('Actual Results')
plt.xlabel('Predicted Results')
ax1.xaxis.set_label_position("top")

profits = TN*1500 + FP*0 - FN*5000 + TP*0

print("At Threshold:", chosen_threshold)
print("Predicted Profit:" ,"£",profits)
print("Model's Accuracy:", (TP+TN)/(TN+TP+FN+FP))
print("Model's Misclassification:", (FP+FN)/(TN+TP+FN+FP))
print("Model's Specificity:", TN/(FP+TN))
print("Model's Sensitivity:", TP/(TP+FN))

## Retraining the model

In [ ]:
# Combining the validation and Training set to create new training set  

trainX_final=pd.concat([TrainX, ValidationX])
trainY_final=pd.concat([TrainY, ValidationY])

print(trainX_final.shape)
print(trainY_final.shape)

In [ ]:
# Normalizing the columns with different orders of magnitude in the testing set

scaler = preprocessing.MinMaxScaler()
TestX[col_norm] = scaler.fit_transform(TestX[col_norm])

In [ ]:
# Retrainong the model with the final training set

logr_final = LogisticRegression(max_iter=1000).fit(trainX_final, trainY_final.values.ravel())

In [ ]:
# Testing the model to the testing set

In [ ]:
# Calculating the probabilities based on the test set
Y_test_probs=logr_final.predict_proba(TestX)[:,1]
# Assigning the selected threshold
threshold = chosen_threshold
# Predicting the values of Y
Y_test_pred=np.where(Y_test_probs > threshold, 1, 0)

In [ ]:
# Creating the ROC curve
fpr, tpr, thresholds = roc_curve(TestY, Y_test_probs)

# Dispaying the ROC curve
fig = plt.figure()
ax = fig.add_subplot(1,1,1)
ax.plot(fpr, tpr, linewidth=3)
plt.show()

# Finding the AUC score of the curve
AUC = roc_auc_score(TestY, Y_test_probs)
print('The AUC score of the ROC curve is:', AUC)   

In [ ]:
# Creating the confusion matrix
cm = confusion_matrix(TestY,Y_test_pred)
TN = cm[0][0]
FP = cm[0][1]
FN = cm[1][0]
TP = cm[1][1]

# Visualizing the Confusion Matrix
fig1 = plt.figure()
ax1 = fig.add_subplot(1,1,1)
sns.heatmap(pd.DataFrame(cm), annot=True, cmap="YlGnBu" ,fmt='g')
plt.ylabel('Actual Results')
plt.xlabel('Predicted Results')
ax1.xaxis.set_label_position("top")

profits = TN*1500 + FP*0 - FN*5000 + TP*0

print("At Threshold:", chosen_threshold)
print("Predicted Profit:" ,"£",profits)
print("Model's Accuracy:", (TP+TN)/(TN+TP+FN+FP))
print("Model's Misclassification:", (FP+FN)/(TN+TP+FN+FP))
print("Model's Specificity:", TN/(FP+TN))
print("Model's Sensitivity:", TP/(TP+FN))

# Using the Model on New Applicants

In [ ]:
df_real = pd.read_csv('New_applicantions.csv')
df_real.T.head(50)

## Pre-prossesing the new data

In [ ]:
# Drop Unnamed: 24 column
df_real.drop(['Unnamed: 24'], axis=1, inplace=True)

In [ ]:
# Replace the values of 5,6 in Education column with 0 because they denote the same Unknown info  
print('Before replace:' , df_real['EDUCATION'].unique())
df_real['EDUCATION'].replace({5 : 0, 6 : 0}, inplace=True)
print('After replace:', df_real['EDUCATION'].unique())

# Make the values of Sex 0 and 1
print('Before reduction:' , df_real['SEX'].unique())
df_real['SEX'] = df_real['SEX']- 1
print('After reduction:', df_real['SEX'].unique())

In [ ]:
for i in range(1,7):
    column = 'PAY_'+str(i)
    ncol = 'DELAY_'+str(i)

    col_num = df_real[column]
    col_num = col_num.replace({-2: 0, -1: 0})
    NUM = pd.DataFrame(list(col_num), columns=[ncol])
    col_cat = df_real[column]
    col_cat = col_cat.replace({2: 1, 3: 1, 4: 1, 5: 1, 6: 1, 7: 1, 8: 1, 9: 1})
    CAT = pd.DataFrame(list(col_cat), columns=[column+'_CATEGORY'])
    if i==1:
        dfNew = pd.concat([NUM,CAT], axis=1)
    else:
        temp = pd.concat([NUM,CAT], axis=1)
        dfNew = pd.concat([dfNew, temp], axis=1)

df_real = pd.concat([df_real, dfNew], axis=1)
df_real = df_real.drop(['PAY_1','PAY_2','PAY_3','PAY_4','PAY_5','PAY_6'], axis=1)

In [ ]:
# Creating Dummies for categorical variables
df_real = pd.get_dummies(df_real,drop_first=True,columns=['EDUCATION','MARRIAGE'])
df_real = pd.get_dummies(df_real,drop_first=True,columns=['PAY_1_CATEGORY','PAY_2_CATEGORY','PAY_3_CATEGORY',
                                                'PAY_4_CATEGORY','PAY_5_CATEGORY','PAY_6_CATEGORY'])

In [ ]:
X = df_real.drop(['ID'],axis=1)
X = X.drop(ex_var,axis=1)

In [ ]:
# Normalizing the columns with different orders of magnitude in the real dataset 
scaler = preprocessing.MinMaxScaler()
X[col_norm] = scaler.fit_transform(X[col_norm])

## Predicting default clients in the New applicants

In [ ]:
X

In [ ]:
# Calculating the probabilities based on the real set
Y_test_probs=logr_final.predict_proba(X)[:,1]
# Assigning the selected threshold
threshold = chosen_threshold
# Predicting the values of Y
Y_test_pred=np.where(Y_test_probs > threshold, 1, 0)

In [ ]:
Y_test_pred

In [ ]:
print('Number of new applicants that defaulted:', np.count_nonzero(Y_test_pred), 'out of 1000.')

In [ ]:
# Creating column for recomended client's for issuing credit

Issue = []
for i in Y_test_pred:
    if i == 0: j=1
    else: j=0
    Issue.append(j)

In [ ]:
Issue = pd.DataFrame({'Issue_Credit': Issue})
Issue.head()

In [ ]:
# Creating new csv with only recomentadion

Issue.to_csv('Recomendation_For_New_Applications.csv',index=False)

## Answering the Questions 2 & 3

In [ ]:
Idx = [i+1 for i in range(len(Y_test_probs))]
client_prob = pd.DataFrame({'Client_index': Idx, 'Prob_to_Default': Y_test_probs}) 
client_prob.head()

Question 2 - Which three of the 1,000 pilot clients are most likely to repay the loan if it were granted to them?

In [ ]:
best_clients = client_prob.sort_values('Prob_to_Default')
best_clients.head(3)

Question 3 - Which three of 1,000 pilot clients are least likely to repay the loan if it were granted to them?

In [ ]:
best_clients.tail(3)